In [253]:
import logging
import time
from datetime import datetime, date, timedelta, timezone
import akshare as ak
import sqlalchemy as db  # Version Check
import pandas as pd
import numpy as np
from numpy import nan
import logging
import sqlalchemy as db 
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData, Table, Sequence, Column, Float, Integer, String
from sqlalchemy.sql import select
from urllib.error import HTTPError, URLError

In [254]:
def str2str(s='Mon Jul 06 11:06:15 2020', strp='%a %b %d %H:%M:%S %Y', strf='%a %b %d %H:%M:%S %Y'):
    return time.strftime(
        strf, 
        time.strptime(
            s, 
            strp)
    )

def str2struct(s, strp='%Y-%m-%d'):
    return time.strptime(s, strp)
    
    
def struct2str(struct=None, strf='%Y-%m-%d', local=True):
    # time.asctime(struct=time.localtime())
    if local:
        return time.strftime(strf, struct or time.localtime())
    else:
        return time.strftime(strf, struct or time.gmtime())


# 参数定义的顺序：必选参数、默认参数、可变参数和关键字参数
def stamp2struct(stamp=None, local=True):
    # time.localtime([secs])方法是将一个时间戳转换为当前时区的struct_time。secs参数未提供，则以当前时间为准。
    # time.gmtime([secs])方法是将一个时间戳转换为UTC时区（0时区）的struct_time。secs参数未提供，则以当前时间为准。
    if local:
        return time.localtime(stamp or time.time())
    else:
        return time.gmtime(stamp or time.time())

def struct2stamp(struct=None, local=True):
    if local:
        return time.mktime(struct or time.localtime())
    else:
        return time.mktime(struct or time.gmtime())

def str2stamp(s, strp='%a %b %d %H:%M:%S %Y', local=True):
    return struct2stamp(str2struct(s, strp), local)

def stamp2str(stamp=None, strf='%a %b %d %H:%M:%S %Y', local=True):
    # time.ctime(stamp=time.time()), 
    return struct2str(stamp2struct(stamp, local), strf, local)

# time.time()：返回当前时间的时间戳。
# time.localtime(seconds=time.time())：返回当前时间的时间元组。
# time.gmtime(seconds=time.time())：返回当前时间的UTC（0时区）时间元组。
# time.asctime(struct=time.localtime())：time.localtime()作为参数传入。返回当前时间元组的字符串'Sun Oct 1 12:04:38 2017'。
# time.ctime(stamp=time.time())：time.time()作为参数传入。返回当前时间戳的字符串'Sun Oct 1 12:04:38 2017'。
# time.sleep(seconds)
# time.strftime(format='%a %b %d %H:%M:%S %Y', struct=time.localtime())
# time.strptime(s, format='%a %b %d %H:%M:%S %Y')
# time.mktime(struct)：将一个struct_time转化为时间戳。
'%a %b %d %H:%M:%S %Y'
print(
    time.strftime('%a %b %d %H:%M:%S %Y'),
    time.strftime('%a %b %d %H:%M:%S %Y', time.localtime()),
    time.strftime('%a %b %d %H:%M:%S %Y', time.gmtime()),
    time.strptime('Mon Jul 06 10:54:18 2020'),
    time.strptime('Mon Jul 06 10:54:18 2020', '%a %b %d %H:%M:%S %Y'),
    sep='\n'
)
# time.strptime('Mon Jul 06 10:54:18 2020')

Mon Jul 06 20:01:46 2020
Mon Jul 06 20:01:46 2020
Mon Jul 06 12:01:46 2020
time.struct_time(tm_year=2020, tm_mon=7, tm_mday=6, tm_hour=10, tm_min=54, tm_sec=18, tm_wday=0, tm_yday=188, tm_isdst=-1)
time.struct_time(tm_year=2020, tm_mon=7, tm_mday=6, tm_hour=10, tm_min=54, tm_sec=18, tm_wday=0, tm_yday=188, tm_isdst=-1)


In [255]:
connection_uri = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}".format(**{
    "user": "root",
    "password": "mysql123456",
    "host": "localhost",
    "port": "3306",
    "dbname": "xinguan"
})
engine = db.create_engine(connection_uri)


In [256]:
metadata = MetaData()

t_daily = Table(
    'csse_daily',
    metadata,
    Column('id', Integer, Sequence('csse_daily_id_seq'), primary_key=True),
    Column('us_only_county_code_fips', Integer),
    Column('us_only_county_name', String(64)),
    Column('higher', String(64)),
    Column('lower', String(64)),
    Column('last_update', String(64)),
    Column('last_update_unify', String(64)),
    Column('report_day', String(64)),
    Column('latitude', Float),
    Column('longitude', Float),
    Column('confirmed', Integer),
    Column('dead', Integer),
    Column('recovered', Integer),
    Column('active', Integer),
    Column('district', String(64)),
)

t_daily.drop(engine, checkfirst=True)
t_daily.create(engine)



In [257]:
retry = 100
report_day = "2020-03-22"
for j in range(retry):
    try:
        df0322 = ak.covid_19_csse_daily(date=report_day)
        logging.info("成功WEB：{0}".format(report_day))
        print("成功WEB：{0}".format(report_day))
        time.sleep(0.1)
        break
    except HTTPError as e:
        print(e)
        logging.info(e)
        print("未来数据当前不可得")
        logging.info("未来数据当前不可得")
        break
    except URLError as e:
        if j == retry - 1:
            logging.info(e)
            logging.info("失败WEB：{0}".format(report_day))
            print("失败WEB：{0}".format(report_day))
        logging.info(e)
        print(e)
        logging.info("重试WEB：{0}".format(report_day))
        print("重试WEB：{0}".format(report_day))


成功WEB：2020-03-22


In [258]:
df0322.rename(columns={
    'Province/State':'higher',
    'Country/Region':'lower',
    'Last Update':'last_update',
    'Confirmed':'confirmed',
    'Deaths':'dead',
    'Recovered':'recovered'
}, inplace=True)


df0322.rename(columns={
    'Province/State':'higher',
    'Country/Region':'lower',
    'Last Update':'last_update',
    'Latitude':'latitude',
    'Longitude':'longitude',
    'Confirmed':'confirmed',
    'Deaths':'dead',
    'Recovered':'recovered'
}, inplace=True)

df0322.rename(columns={
    'FIPS': 'us_only_county_code_fips',
    'Admin2': 'us_only_county_name',
    'Province_State':'higher',
    'Country_Region':'lower',
    'Last_Update':'last_update',
    'Lat':'latitude',
    'Long_':'longitude',
    'Confirmed':'confirmed',
    'Deaths':'dead',
    'Recovered':'recovered',
    'Active':'active',
    'Combined_Key':'district'
}, inplace=True)


In [308]:


try:
    df0322['last_update_unify'] = df0322['last_update'].map(lambda x:str2str(x, '%m/%d/%Y %H:%M', '%Y-%m-%dT%H:%M:%S'))     
    # df0122['report_day'] = df0122['last_update'].map(lambda x:str2str(x, '%m/%d/%Y %H:%M', '%Y-%m-%d')) 
    df0322['report_day'] = report_day
except Exception as e:
    print(e)
    pass

try:
    df0322['last_update_unify'] = df0322['last_update'].map(lambda x:str2str(x, '%Y-%m-%dT%H:%M:%S', '%Y-%m-%dT%H:%M:%S'))     
    # df0301['report_day'] = df0301['last_update'].map(lambda x:str2str(x, '%Y-%m-%dT%H:%M:%S', '%Y-%m-%d')) 
    df0322['report_day'] = report_day
except Exception as e:
    print(e)
    pass

try:
    df0322['last_update_unify'] = df0322['last_update'].map(lambda x:str2str(x, '%m/%d/%y %H:%M', '%Y-%m-%dT%H:%M:%S'))     
    df0322['report_day'] = df0322['last_update'].map(lambda x:str2str(x, '%m/%d/%y %H:%M', '%Y-%m-%d')) 
except Exception as e:
    print(e)
    pass





df0322.fillna(value={
    'us_only_county_code_fips':0,
    'us_only_county_name':'null',
    'higher':'null',
    'lower':'null',
    'district':'null',
    'latitude':0,
    'longitude':0,
    'confirmed':0,
    'dead':0,
    'recovered':0,
    'active':0,    
}, inplace=True)
df0322


time data '3/22/20 23:45' does not match format '%m/%d/%Y %H:%M'
time data '3/22/20 23:45' does not match format '%Y-%m-%dT%H:%M:%S'


,us_only_county_code_fips,us_only_county_name,higher,lower,last_update,latitude,longitude,confirmed,dead,recovered,active,district,last_update_unify,report_day
0,36061.0,New York City,New York,US,3/22/20 23:45,40.767273,-73.971526,9654,63,0,0,"New York City, New York, US",2020-03-22T23:45:00,2020-03-22
1,36059.0,Nassau,New York,US,3/22/20 23:45,40.740665,-73.589419,1900,4,0,0,"Nassau, New York, US",2020-03-22T23:45:00,2020-03-22
2,36119.0,Westchester,New York,US,3/22/20 23:45,41.162784,-73.757417,1873,0,0,0,"Westchester, New York, US",2020-03-22T23:45:00,2020-03-22
3,36103.0,Suffolk,New York,US,3/22/20 23:45,40.883201,-72.801217,1034,9,0,0,"Suffolk, New York, US",2020-03-22T23:45:00,2020-03-22
4,36087.0,Rockland,New York,US,3/22/20 23:45,41.150279,-74.025605,455,1,0,0,"Rockland, New York, US",2020-03-22T23:45:00,2020-03-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,0.0,null,null,Syria,3/22/20 23:45,34.802075,38.996815,1,0,0,1,Syria,2020-03-22T23:45:00,2020-03-22
3413,0.0,null,null,Timor-Leste,3/22/20 23:45,-8.874217,125.727539,1,0,0,1,Timor-Leste,2020-03-22T23:45:00,2020-03-22
3414,0.0,null,null,Uganda,3/22/20 23:45,1.373333,32.290275,1,0,0,1,Uganda,2020-03-22T23:45:00,2020-03-22
3415,11001.0,District of Columbia,District of Columbia,US,3/22/20 23:45,38.904178,-77.016560,102,2,0,0,"District of Columbia,District of Columbia,US",2020-03-22T23:45:00,2020-03-22


In [261]:
for j in range(retry):
    try:
        df0322.to_sql(
            t_daily.name,
            con=engine,
            if_exists='append',
            index=False)
        logging.info("成功DB：{0}".format(report_day))
        print("成功DB：{0}".format(report_day))
        time.sleep(0.1)
        break
    except Exception as e:
        if j == retry - 1:
            logging.info("失败DB：{0}".format(report_day))
            print("失败DB：{0}".format(report_day))
            break
        print(e)
        logging.info("重试DB：{0}".format(report_day))
        print("重试DB：{0}".format(report_day))
        logging.info(e)


成功DB：2020-03-22


* `>= 01-22-2020.csv`
* `>= 03-01-2020.csv`
* `>= 03-22-2020.csv`

***


* FIPS: US only. FederalInformationProcessingStandards. Code that uniquely identifies counties within the USA.
* Admin2: US only. County name. 
* Province_State: Province, state or dependency name. 
    * Province/State: 
        * China - province name; 
        * US/Canada/Australia/ - city name, state/province name; 
        * Others - name of the event (e.g., "Diamond Princess" cruise ship); 
        * other countries - blank.
* Country_Region: 
    * Country, region or sovereignty name. The names of locations included on the Website correspond with the official designations used by the U.S. Department of State.
* Last Update: MM/DD/YYYY HH:mm:ss (24 hour format, in UTC).
* Lat and Long_: Dot locations on the dashboard. All points (except for Australia) shown on the map are based on geographic centroids, and are not representative of a specific address, building or any location at a spatial scale finer than a province/state. Australian dots are located at the centroid of the largest city in each state.
* Confirmed: Confirmed cases include `presumptive positive` cases and `probable` cases, in accordance with CDC guidelines as of April 14.
* Deaths: Death totals in the US include confirmed and probable, in accordance with CDC guidelines as of April 14.
* Recovered: Recovered cases outside China are estimates based on local media reports, and state and local reporting when available, and therefore may be substantially lower than the true number. US state-level recovered cases are from COVID Tracking Project.
* Active: Active cases = total confirmed - total recovered - total deaths.
* Incidence_Rate: Admin2 + Province_State + Country_Region.
* Case-Fatality Ratio (%): = confirmed cases per 100,000 persons.
* US Testing Rate: = total test results per 100,000 persons. The "total test results" is equal to "Total test results (Positive + Negative)" from COVID Tracking Project.
* US Hospitalization Rate (%): = Total number hospitalized / Number confirmed cases. The "Total number hospitalized" is the "Hospitalized – Cumulative" count from COVID Tracking Project. The "hospitalization rate" and "hospitalized - Cumulative" data is only presented for those states which provide cumulative hospital data.

***

Update frequency:
* Files on and after April 23, once per day between 03:30 and 04:00 UTC.
* Files from February 2 to April 22: once per day around 23:59 UTC.
* Files on and before February 1: the last updated files before 23:59 UTC. Sources: archived_data and dashboard.


In [108]:
conn = engine.connect()

In [112]:
sql = select([t_daily.c.report_day],distinct=True)
r = conn.execute(sql)
r = list(r)

In [113]:
# 结果是迭代器只能用一次，而django的orm惰性加载list
[tstr[0] for tstr in r]

['2020-03-22',
 '2020-03-21',
 '2020-03-20',
 '2020-03-19',
 '2020-03-18',
 '2020-03-16',
 '2020-03-14',
 '2020-03-13',
 '2020-03-12',
 '2020-03-08',
 '2020-02-23',
 '2020-02-12']

In [205]:
lte_pre = max([tstr[0] for tstr in r], key=lambda x:str2struct(x))
lte_pre = str2stamp(lte_pre, strp='%Y-%m-%d')
lte_pre = datetime.fromtimestamp(lte_pre)

lte_now = time.time()
datetime.fromtimestamp(lte_now) 
lte_now = datetime.now() 

def synchronize_csse_daily(engine, manner='append', endurance=15):
    retry = 100
    conn = engine.connect()
    
    if manner == 'init':
        t_daily.drop(engine, checkfirst=True)
        t_daily.create(engine)

    sql = select([t_daily.c.report_day], distinct=True)
    r = conn.execute(sql)  # iterator
    r = list(r)
    r = r or ['2020-1-21']

    lte_pre = max([tstr[0] for tstr in r], key=lambda x:str2struct(x)) or 
    lte_pre = str2stamp(lte_pre, strp='%Y-%m-%d')
    lte_pre = datetime.fromtimestamp(lte_pre)

    lte_now = datetime.now()

    dfs = []
    while lte_pre <= lte_now:
        lte_pre += timedelta(days=1)

        # 获取数据
        report_day = lte_pre
        df = None
        for j in range(retry):
            try:
                df = ak.covid_19_csse_daily(date=report_day)
                logging.info("成功WEB：{0}".format(report_day))
                time.sleep(0.1)
                break
            except HTTPError as e:
                raise Exception("未来数据当前不可得")
            except URLError as e:
                if j == retry - 1:
                    logging.info(e)
                    raise Exception("失败WEB：{0}".format(report_day))
                logging.info(e)
                logging.info("重试WEB：{0}".format(report_day))

        # 修改字段
        df.rename(columns={
            'Province/State':'higher',
            'Country/Region':'lower',
            'Last Update':'last_update',
            'Confirmed':'confirmed',
            'Deaths':'dead',
            'Recovered':'recovered'
        }, inplace=True)

        df.rename(columns={
            'Province/State':'higher',
            'Country/Region':'lower',
            'Last Update':'last_update',
            'Latitude':'latitude',
            'Longitude':'longitude',
            'Confirmed':'confirmed',
            'Deaths':'dead',
            'Recovered':'recovered'
        }, inplace=True)

        df.rename(columns={
            'FIPS': 'us_only_county_code_fips',
            'Admin2': 'us_only_county_name',
            'Province_State':'higher',
            'Country_Region':'lower',
            'Last_Update':'last_update',
            'Lat':'latitude',
            'Long_':'longitude',
            'Confirmed':'confirmed',
            'Deaths':'dead',
            'Recovered':'recovered',
            'Active':'active',
            'Combined_Key':'district'
        }, inplace=True)
        
        # 统一DataFrame结构
        df = df.reindex(columns=[c.name for c in t_daily.columns])
        
        
        
        # 处理空值
        df.fillna(value={
            'us_only_county_code_fips':0,
            'us_only_county_name':'null',
            'higher':'null',
            'lower':'null',
            'district':'null',
            'latitude':0,
            'longitude':0,
            'confirmed':0,
            'dead':0,
            'recovered':0,
            'active':0,    
        }, inplace=True)
        
        
        # 数据持久化
        dfs.append(df)
        if len(dfs) == endurance:
            for j in range(retry):
                try:
                    pd.concat(dfs).to_sql(
                        t_daily.name,
                        con=engine,
                        if_exists='append',
                        index=False
                    )
                    logging.info("成功DB：{0},{1},{2}".format(province, city, district))
                    time.sleep(0.1)
                    break
                except Exception as e:
                    if j == retry-1:
                        raise Exception("失败DB：{0},{1},{2}".format(province, city, district))
                    logging.info(e)
                    logging.info("重试DB：{0},{1},{2}".format(province, city, district))
            logging.info(endurance)
            time.sleep(0.3)
            dfs = []
        for j in range(retry):
            try:
                pd.concat(dfs).to_sql(
                    t_daily.name,
                    con=engine,
                    if_exists='append',
                    index=False
                )
                logging.info("成功DB：{0},{1},{2}".format(province, city, district))
                time.sleep(0.1)
                break
            except Exception as e:
                if j == retry - 1:
                    raise Exception("失败DB：{0},{1},{2}".format(province, city, district))
                logging.info(e)
                logging.info("重试DB：{0},{1},{2}".format(province, city, district))
            
            
# users.delete().where(t_daily.c.report_day == lte)

2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-13
2020-06-14
2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
2020-06-20

In [163]:
datetime.fromtimestamp(lte_now), datetime.fromtimestamp(lte_pre)
datetime.strftime(datetime.fromtimestamp(lte_pre) + timedelta(days=1), format='%Y-%m-%d')



'2020-03-23'

In [316]:
# datetime replace 时区也可以转换
# datetime和time之间的通道是timestamp
print(
    datetime(2018, 3, 12, 14, 52, 35, 276589),
    type(datetime(2018, 3, 12, 14, 52, 35, 276589)),
    datetime(2018, 3, 12, 14, 52, 35, 276589).replace(2020, 7, 1),
    type(datetime(2018, 3, 12, 14, 52, 35, 276589).replace(2020, 7, 1)),
    datetime.fromtimestamp(time.time()),
    type(datetime.fromtimestamp(time.time())),
    datetime.now(),
    type(datetime.now()),
    datetime.today(),
    type(datetime.today()),
    datetime.today().now().today().now().now().today().today(),
    type(datetime.today().now().today().now().now().today().today()),
    datetime.now().time(),  # time方法类似于实例方法，必须由实例来调用
    type(datetime.now().time()),
    datetime.now().date(),  # time方法类似于实例方法，必须由实例来调用
    type(datetime.now().date()),
    datetime.now().timestamp(),  # time方法类似于实例方法，必须由实例来调用
    type(datetime.now().timestamp()),
    datetime.now().timetuple(),
    type(datetime.now().timetuple()),
    datetime.now().year,
    type(datetime.now().year),
    datetime.now().month,
    type(datetime.now().month),
    timedelta(days=1),
    type(timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)),
    # 天数不定的年（计时单位没有影响，关键在于不定），天数不定的月，都不可以作为timedelta的构造参数
    timedelta(days=1) + timedelta(days=2) ,
    type(timedelta(days=1) + timedelta(days=2)),
    timedelta(days=1) + datetime.now(),
    type(timedelta(days=1) + datetime.now()),
    sep='\n'
)

# date
# calendar
# 以datetime、timedelta作为基本单位的运算

datetime.today()

2018-03-12 14:52:35.276589
<class 'datetime.datetime'>
2020-07-01 14:52:35.276589
<class 'datetime.datetime'>
2020-07-07 00:19:21.556648
<class 'datetime.datetime'>
2020-07-07 00:19:21.556657
<class 'datetime.datetime'>
2020-07-07 00:19:21.556667
<class 'datetime.datetime'>
2020-07-07 00:19:21.556682
<class 'datetime.datetime'>
00:19:21.556695
<class 'datetime.time'>
2020-07-07
<class 'datetime.date'>
1594052361.556701
<class 'float'>
time.struct_time(tm_year=2020, tm_mon=7, tm_mday=7, tm_hour=0, tm_min=19, tm_sec=21, tm_wday=1, tm_yday=189, tm_isdst=-1)
<class 'time.struct_time'>
2020
<class 'int'>
7
<class 'int'>
1 day, 0:00:00
<class 'datetime.timedelta'>
3 days, 0:00:00
<class 'datetime.timedelta'>
2020-07-08 00:19:21.556735
<class 'datetime.datetime'>


datetime.datetime(2020, 7, 7, 0, 19, 21, 557829)

In [313]:
# Python中定义的实例方法？ 类方法？ 在函数中定义的函数



In [326]:
print(
    date.today(),
    type(date.today()),
    date.fromtimestamp(time.time()),
    type(date.fromtimestamp(time.time())),
    date(2999, 11, 30),
    type(date(2999, 11, 30)),
    date(2020, 7, 1).replace(1997, 4, 15),
    type(date(2020, 7, 1).replace(1997, 4, 15)),
    date.min,
    date.max,
    date.resolution,
    type(date.resolution),
    date(2999, 11, 30).year,
    date(2999, 11, 30).month,  # month的范围是[1, 12]
    date(2999, 11, 30).day,
    sep='\n'
)


2020-07-07
<class 'datetime.date'>
2020-07-07
<class 'datetime.date'>
2999-11-30
<class 'datetime.date'>
1997-04-15
<class 'datetime.date'>
0001-01-01
9999-12-31
1 day, 0:00:00
<class 'datetime.timedelta'>
2999
11
30


In [238]:
df = pd.DataFrame({"A": [1,2,3], "B": [2,3,4]})
df


,A,B
0,1,2
1,2,3
2,3,4


In [250]:

r = r or ['2020-01-21']

lte_pre = max([tstr[0] for tstr in r], key=lambda x: str2struct(x, strp='%Y-%m-%d'))
logging.info(lte_pre, type(lte_pre))
lte_pre = str2stamp(lte_pre, strp='%Y-%m-%d')
lte_pre = datetime.fromtimestamp(lte_pre)

lte_pre

datetime.datetime(2020, 3, 22, 0, 0)

In [297]:

connection_uri = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}".format(**{
    "user": "root",
    "password": "mysql123456",
    "host": "localhost",
    "port": "3306",
    "dbname": "xinguan"
})
engine = db.create_engine(connection_uri)
conn = engine.connect()
sql = select([t_daily.c.report_day], distinct=True)
r = conn.execute(sql)  # iterator

In [298]:
r = [row.report_day for row in r if row.report_day is not None]

In [305]:
r = r or ['2020-01-21']


In [306]:
lte_pre = max(r, key=lambda x: str2struct(x, strp='%Y-%m-%d'))
lte_pre = str2stamp(lte_pre, strp='%Y-%m-%d')
lte_pre = datetime.fromtimestamp(lte_pre)

In [307]:
lte_pre

datetime.datetime(2020, 1, 21, 0, 0)

In [ ]:
date.replace(year, month, day):返回一个相同值的data对象, 除了这些参数给关键字指定新的值.
date.timetuple(): 返回一个time.struct_time对象.
date.toordinal(): 返回一个Gregoian Calendar对象.
date.weekday(): 返回day of the week. 星期一为0,星期日为6.
date.isoweekday(): 返回day of the week. 星期一为1,星期日为7.
date.isocalendar(): 返回一个三元组, (ISO year, ISO week number, ISO weekday).
date.isoformat(): 返回 一个'YYYY-MM-DD'的字符串格式.
date.ctime(): 返回一个字符串日期, d.ctime() 等同于 time.ctime(time.mktime(d.timetuple())).
date.strftime(format): 返回一个字符串日期, 格式自定义.

In [ ]:
datetime.date：表示日期的类。常用的属性有year, month, day；
datetime.time：表示时间的类。常用的属性有hour, minute, second, microsecond；
datetime.datetime：表示日期时间。
datetime.timedelta：表示时间间隔，即两个时间点之间的长度。
datetime.tzinfo：与时区有关的相关信息。（这里不详细充分讨论该类，感兴趣的童鞋可以参考python手册）